使用没用预训练的resnet网络

In [5]:
import os
import mat4py
from PIL import Image
import numpy as np
import torch
from torch import optim, nn
from torch.utils.data.dataset import Dataset
from torchvision import transforms, models
import helper

In [6]:
trainDir = '../datasets/DukeMTMC-reID/bounding_box_train'
testDir = '../datasets/DukeMTMC-reID/bounding_box_test'

In [7]:
trains = os.listdir(trainDir)
tests = os.listdir(testDir)

In [8]:
len(trains), len(tests)

(16522, 17661)

In [9]:
Labels = mat4py.loadmat('../datasets/DukeMTMC-reID/duke_attribute.mat')
Labels = Labels['duke_attribute']
trainLabels = Labels['train']
testLabels = Labels['test']
keys = trainLabels.keys()

In [10]:
class MyDataSet(Dataset):
    def __init__(self, pathOfImgs, Labels, keys, dir):
        self.imgs = pathOfImgs
        self.labels = Labels
        self.transforms = transforms.Compose([transforms.Resize((224, 224)), 
                                              transforms.ToTensor(),
                                              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
        #self.loader = loader
        self.keys = keys
        self.dir = dir
                                             
    def __getitem__(self, index):
        img_path = self.imgs[index]
        #print(img_path)
        img = Image.open(self.dir + '/' + img_path)
        #print(img.size)
        trans = transforms.Compose([transforms.ToTensor(),])
        img = self.transforms(img)
        #print(1)
        #print(img.shape)
        img_id = img_path[0:4]
        #print(self.labels.keys())
        img_idx_in_Labels = self.labels['image_index'].index(img_id)
        labels = []
        for key in keys:
            if key == 'image_index':
                 break
            labels.append(self.labels[key][img_idx_in_Labels])
        labels = np.array(labels)
        labels = torch.from_numpy(labels) - 1
        labels = labels.float()
        return (img, labels)
        
    def __len__(self):
        return len(self.imgs)
        
        
trainDataSet = MyDataSet(trains, trainLabels, keys, trainDir)
trainLoader = torch.utils.data.DataLoader(trainDataSet, batch_size=32, shuffle=True)

In [11]:
resnet50 = models.resnet50(pretrained=False)
#resnet50.load_state_dict(torch.load('../models/resnet50.pth'))

In [12]:
# Freeze parameters so we don't backprop through them
# for param in resnet50.parameters():
#     param.requires_grad = False

from collections import OrderedDict
from torch import nn
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 1024)),
                          ('relu1', nn.ReLU()),
                          ('fc2', nn.Linear(1024, 512)),
                          ('relu2', nn.ReLU()),
                          ('fc3', nn.Linear(512, 23)),
                          ('output', nn.Sigmoid())
                          ]))

In [13]:
resnet50.fc = classifier

In [14]:
def errFun(res, label):
  y = label * torch.log(res) + (1-label) * torch.log(1-res)
  return -torch.sum(y) / y.numel()

In [15]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=2)

In [16]:
model = resnet50.to(device)
# for param in model.parameters():
#     print(param.requires_grad)

In [ ]:
optimizer = optim.Adagrad(model.parameters(), lr=0.001)
epoch = 5
for ep in range(epoch):
  running_err = 0
  i = 0
  for img, label in trainLoader:
    img = img.to(device)
    label = label.to(device)
    optimizer.zero_grad()
    output = model(img)
    err = errFun(output, label)
    err.backward()
    optimizer.step()
    running_err += err.item() * label.numel()
    i += 1
    print(i)
    if i%100 == 0:
      print(running_err / ((i+1) * len(img)))
  print('err of epoch{}: {}'.format(ep+1, running_err/len(trainDataSet)))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
8.047546740510676
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
7.6053442342660915
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268

In [17]:
# torch.save(model.to(torch.device('cpu')).state_dict(), '/content/gdrive/My Drive/fc.pth')
torch.save(model.to(torch.device('cpu')).state_dict(), '../TrainedResNet.pth')

In [ ]:
testDataset = MyDataSet(tests, testLabels, keys, testDir)
testLoader = torch.utils.data.DataLoader(testDataset, batch_size=32, shuffle=True)

In [ ]:
testErr = 0
testAcc = 0
count = 0
for img, label in testLoader:
  img, label = img.to(device), label.float().to(device)
  pred = model(img)
  err = errFun(pred, label)
  testErr += err.item() * len(img)
  for i in range(pred.shape[0]):
    for j in range(pred.shape[1]):
      pred[i,j] = 1.0 if pred[i,j] > 0.5 else 0.0
  difference = torch.abs(pred - label)
  testAcc += torch.mean(torch.sum(difference, dim=0) / difference.shape[0]).item()
  count += 1
testAcc /= count
testErr /= len(testDataset)